#                             ## PCA with PYOPENCL ##

## Introduction

 # Technology choice
 
The following project uses PYOPENCL to parallelize computations on the GPU, more precisely on the computer's iGPU.
I chose PYOPENCL knowing that I don't have an NVIDIA card, and because of this it is not possible to use PYCUDA.
After taking the prebuilt binary corresponding to my operating system and python's version. I made the installation Nous avons by pip install on the Anaconda prompt. Note that I did'nt need to download the opencl driver because it is integrated in the computer.

 # Topic choice
 
 I chose to program a PCA with pyopencl, the final objective is to obtain the variance covariance matrix that contains proper values in its' diagonal that represent the empirical variances of each axes, from the one who explains it the most to the last one.
 The goal of a PCA is to project data on a high dimension by reducting to the 2 dimensions that explains the most the data.


# Environment preparation

In [1]:
# Import of the necessary libraries
import time
import pyopencl as cl
import numpy as np
from __future__ import division
from numpy import matlib 
import datetime
import pyopencl.array
import os


In [3]:
os.environ['PYOPENCL_COMPILER_OUTPUT'] = '1'

In [4]:
NAME = 'Intel(R) OpenCL' 
platformes = cl.get_platforms()
devs = None
for platform in platformes:
    if platform.name == NAME:
        devs = platform.get_devices()
ctx = cl.Context(devs)
queue = cl.CommandQueue(ctx)


# Dataset preparation 

In [5]:
# Creation of the matrix of size (width=25000,height=40) full of random values 

width = np.int32(25000)
height = np.int32(40)
a = np.random.randint(1, 140, size=(width, height)).astype(np.float32)


In [6]:
D=np.eye(width, k=0).astype(np.float32)
D=D*1/width

In [7]:
diag=np.diag(D).astype(np.float32)

   # PCA setting

## Transformation of the sample

 Here I need to transpose the a matrix in order to have $a^T$, then I have to transpose the weight vector in order to have  $D^T$ to compute the gravity center of the scatter plot $W=a^T*D$ 

In [8]:

  #Transposition of the weight diagonal vector

#PYOPENCL
kernel1 = """
__kernel void func1(__global float* diag, __global float* diagT, const unsigned int U,const unsigned int V) {
    unsigned int i = get_global_id(0);
    unsigned int j = get_global_id(1);
    diagT[j + V * i] = diag[i + U * j];
}

// end of the transposition """


U = np.int32(25000) # Vector's size for the transposition 
V=np.int32(1)
diag=np.diag(D).astype(np.float32)
diagT= np.zeros([ U,V ]).astype(np.float32)
mf = cl.mem_flags
diag_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=diag )
diagT_buf = cl.Buffer(ctx, mf.WRITE_ONLY, diagT.nbytes)

#Computation duration

prg1 = cl.Program(ctx, kernel1).build()
t1 = datetime.datetime.now()
prg1.func1(queue, diagT.shape, None, diag_buf, diagT_buf, U,V)
diagT = np.zeros([U, V]).astype(np.float32)
cl.enqueue_copy(queue, diagT, diagT_buf)
delta_t_GPU_transpo_diag = datetime.datetime.now() - t1

### PYTHON
t2 = datetime.datetime.now()
diagT_py=np.transpose(diag)
delta_t_CPU_transpo_diag = datetime.datetime.now() - t2

    #Transposition of the a matrix

### PYOPENCL
kernel2 = """ 
__kernel void func2(__global int* a, __global int* b, const unsigned int width, const unsigned int height) {
    unsigned int i = get_global_id(0);
    unsigned int j = get_global_id(1);
    b[j + width * i] = a[i + height * j];
} 

// end of the transposition """

b = np.zeros([height, width]).astype(np.float32)
mf = cl.mem_flags
a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a )
b_buf = cl.Buffer(ctx, mf.WRITE_ONLY, a.nbytes)

#Computation duration

prg2 = cl.Program(ctx, kernel2).build()
t3 = datetime.datetime.now()
prg2.func2(queue, b.shape, None, a_buf, b_buf, width, height)
b = np.zeros([height, width]).astype(np.float32)
cl.enqueue_copy(queue, b, b_buf)
delta_t_GPU_transpo_a = datetime.datetime.now() - t3


### PYTHON
t4 = datetime.datetime.now()
b_py=np.transpose(a)
delta_t_CPU_transpo_a = datetime.datetime.now() - t4

## Computations durations results

print(delta_t_GPU_transpo_diag)
print(delta_t_CPU_transpo_diag)
print(delta_t_GPU_transpo_a) 
print(delta_t_CPU_transpo_a)



C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <func1> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

0:00:00.031253
0:00:00
0:00:00.062507
0:00:00


We can notice that the transposition of the a matrix with the GPU Nous pouvons noter que la transposition de la matrice a avec la GPU takes a little bit more time thant the other operations.
Now, let us calculate the gravity center W, it is the product $a^TD$, the average vector of each variables


In [9]:
 ### Multiplication of the initial matrix transposed b and the diagonal vector
    
 ## PYOPENCL

kernel3= """__kernel void gemv(const __global float* b,

                                    const __global float* diagT,

                                    uint widthnew, uint heightnew,

                                    __global float* W)
{ // Row index

    uint y = get_global_id(0);
    if (y < heightnew) {
        // Row pointer
        const __global float* row = b + y * widthnew;
        // Compute dot product  
        float dotProduct = 0;
        for (int x = 0; x < widthnew; ++x)
            dotProduct += row[x] * diagT[x];
        // Write result to global memory
        W[y] = dotProduct;
    }
}
"""

widthnew=height
heightnew=width
W = np.zeros([widthnew, 1]).astype(np.float32)
mf = cl.mem_flags
b_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=b)
diagT_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=diagT)
W_buf = cl.Buffer(ctx, mf.WRITE_ONLY, heightnew*widthnew*4)
prg3 = cl.Program(ctx, kernel3).build()

t5 = datetime.datetime.now()
prg3.gemv(queue, W.shape, None
, b_buf
, diagT_buf
, widthnew
, heightnew, W_buf)


W = np.empty([widthnew, 1]).astype(np.float32)
cl.enqueue_copy(queue, W, W_buf)
delta_t_GPU_mul_1 = datetime.datetime.now() - t5
print(delta_t_GPU_mul_1)

# PYTHON
t6 = datetime.datetime.now()
W=b.dot(diagT)
delta_t_CPU_mul_1 = datetime.datetime.now() - t6
print(delta_t_CPU_mul_1)

0:00:00.015628
0:00:00.062507


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <gemv> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compiler

In this operation, we can notice that the GPU has more performance than the CPU. 
I now need to center the matrix on the gravity center like $a-W^T$, so I need to transpose the gravity center vector before

In [10]:
    #Transposition of the gravity center
    
#PYOPENCL    
U = np.int32(1)
V=np.int32(40)

#I use the same kernel thant the one for the previous transposition
WT= np.zeros([ U,V ]).astype(np.float32)
mf = cl.mem_flags
W_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=W )
WT_buf = cl.Buffer(ctx, mf.WRITE_ONLY, WT.nbytes)

prg2 = cl.Program(ctx, kernel2).build()

t7 = datetime.datetime.now()
prg2.func2(queue, WT.shape, None, W_buf, WT_buf, U,V)
WT = np.zeros([U, V]).astype(np.float32)
cl.enqueue_copy(queue, WT, WT_buf)
delta_t_GPU_transpo_W = datetime.datetime.now() - t7

#PYTHON 
t8 = datetime.datetime.now()
WT_CPU=np.transpose(W)
delta_t_CPU_transpo_W = datetime.datetime.now() - t8

print(delta_t_GPU_transpo_W)
print(delta_t_CPU_transpo_W)

0:00:00.015619
0:00:00


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <func2> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

We can notice that, for the transposition , once again, the GPU and the CPU are equally efficient.
Now that I have the gravity center transposed, I need to center the variables by their average like $a - W^T$

In [12]:
  ###Soustraction of the initial matrix by the transposed gravity center vector. This produces a centered matrix 
    
#PYOPENCL

a_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a )
WT_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=WT )
Centre_buf=cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=a )
Centre=np.zeros([width,height]).astype(np.float32)
prg3 = cl.Program(ctx, """
    __kernel void sub(const unsigned int size, __global float * a, __global float * WT, __global float * Centre) {

    int i = get_global_id(1); 
    int j = get_global_id(0);

    Centre[i + size * j] = 0;

    for (int k = 0; k < size; k++)
    {
       
     Centre[i + size * j] += a[k + size * i] - WT[j + size * k];

    }

    }
    """).build()

t9 = datetime.datetime.now()
prg3.sub(queue, a.shape, None,np.int32(len(a)) ,a_buf, WT_buf, Centre_buf)
cl.enqueue_copy(queue, Centre, Centre_buf)
delta_t_GPU_Centré = datetime.datetime.now() - t9


#CPU

t10 = datetime.datetime.now()
Centre_CPU=a-WT
delta_t_CPU_Centré = datetime.datetime.now() - t10

print(delta_t_GPU_Centré)
print(delta_t_CPU_Centré)

C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <sub> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, CompilerW

0:00:32.270290
0:00:00.015629


Here we can notice that the computation duration on the GPU is a lot more important than the CPU.
Now, I have to compute the variance covariance matrix such as $Centre^T*D*Centre$

For this I firstly need to transpose the centered matrix and then apply the multiplication

## Variance covariance matrix computation

In [13]:
  ## Transposition of the centered matrix

#PYOPENCL    
kernel4 = """ 
__kernel void func3(__global int* a, __global int* b, const unsigned int width, const unsigned int height) {
    unsigned int i = get_global_id(0);
    unsigned int j = get_global_id(1);
    b[j + width * i] = a[i + height * j];
} 

// end of the transposition """   

Centre_T = np.zeros([height, width]).astype(np.float32)
mf = cl.mem_flags
Centre_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Centre )
Centre_T_buf = cl.Buffer(ctx, mf.WRITE_ONLY, Centre.nbytes)


prg5 = cl.Program(ctx, kernel4).build()

t11 = datetime.datetime.now()
prg5.func3(queue, Centre_T.shape, None, Centre_buf, Centre_T_buf, width, height)
Centre_T = np.zeros([height, width]).astype(np.float32)
cl.enqueue_copy(queue, Centre_T, Centre_T_buf)
delta_t_GPU_Centré_Transpo = datetime.datetime.now() - t11

#CPU
t12 = datetime.datetime.now()
Centre_T_CPU= np.transpose(Centre)
delta_t_CPU_Centré_Transpo = datetime.datetime.now() - t12

print(delta_t_GPU_Centré_Transpo)
print(delta_t_CPU_Centré_Transpo)


0:00:00.109388
0:00:00


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <func3> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

We notice here a much better computation duration for the CPU than the GPU.
Let us proceed now to the multiplication for the Variance covariance matrix, I split this calculation by a multiplication between the transposed center matrix and D vector. The intermidiate vector produced by this is of a size (40,1) and the final resul, the variance covariance matrix is a square matrix (40,40) 

In [20]:
# Computation of the variance covariance matrix

#1st part

#PYOPENCL
#Let us use the Kernel3 of mtrix vector multiplication 

widthnew=height
heightnew=width
V1 = np.zeros([widthnew, 1]).astype(np.float32)
mf = cl.mem_flags
Centre_T_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Centre_T)
diagT_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=diagT)
V1_buf = cl.Buffer(ctx, mf.WRITE_ONLY, heightnew*widthnew*4)
prg6 = cl.Program(ctx, kernel3).build()


prg6.gemv(queue, V1.shape, None
, Centre_T_buf
, diagT_buf
, widthnew
, heightnew, V1_buf)

V1 = np.empty([widthnew, 1]).astype(np.float32)
t13 = datetime.datetime.now()
cl.enqueue_copy(queue, V1, V1_buf)
delta_t_GPU_mul_2 = datetime.datetime.now() - t13


#CPU
t14 = datetime.datetime.now()
V1= Centre_T.dot(diagT)
delta_t_CPU_mul_2 = datetime.datetime.now() - t14

print(delta_t_GPU_mul_2)
print(delta_t_CPU_mul_2)

0:00:00
0:00:00


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <gemv> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compiler

We can notice that the computation duration is equivalent for both the GPU and CPU. Let use move to the second part of the calculation. We have a vector of size (40,1), once the multiplication with the centered matrix size (25000,40) done, we will obtain a square matrix 

In [19]:
#2nd part

#PYOPENCL

kernel5= """__kernel void gemv2(const __global float* V1,

                                    const __global float* CENTRE,

                                    uint widthnew, uint heightnew,

                                    __global float* V)
{ // Row index

    uint y = get_global_id(0);
    if (y < heightnew) {
        // Row pointer
        const __global float* row = V1 + y * widthnew;
        // Compute dot product  
        float dotProduct = 0;
        for (int x = 0; x < widthnew; ++x)
            dotProduct += row[x] * CENTRE[x];
        // Write result to global memory
        V[y] = dotProduct;
    }
}
"""
#I set the new row size equal to the old column size and the new column size equal to the old row size
widthnew=height
heightnew=width
V = np.zeros([widthnew, widthnew]).astype(np.float32)
mf = cl.mem_flags
V1_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=V1)
Centre_buf = cl.Buffer(ctx, mf.READ_ONLY | mf.COPY_HOST_PTR, hostbuf=Centre)
V_buf = cl.Buffer(ctx, mf.WRITE_ONLY, widthnew*widthnew*4)
prg7 = cl.Program(ctx, kernel5).build()

t15 = datetime.datetime.now()
prg7.gemv2(queue, V.shape, None
, V1_buf
, Centre_buf
, widthnew
, heightnew, V_buf)


V = np.empty([widthnew, widthnew]).astype(np.float32)
cl.enqueue_copy(queue, V, V_buf)
delta_t_GPU_mul_3 = datetime.datetime.now() - t15

print(delta_t_GPU_mul_3)
print(V.shape)

# Multiplication on the CPU
test= V1.dot(Centre)

0:00:00.015629
(40, 40)


C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: Built kernel retrieved from cache. Original from-source build had warnings:
Build on <pyopencl.Device 'Intel(R) HD Graphics 520' on 'Intel(R) OpenCL' at 0x1a552200e80> succeeded, but said:

fcl build 1 succeeded.
bcl build succeeded.
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Compilation started
Compilation done
Linking started
Linking done
Device build started
Device build done
Kernel <gemv2> was successfully vectorized (8)
Done.
  warn(text, CompilerWarning)
C:\Users\MKB\Anaconda3\lib\site-packages\pyopencl\cffi_cl.py:1517: CompilerWarning: From-binary build succeeded, but resulted in non-empty logs:
Build on <pyopencl.Device 'Intel(R) Core(TM) i7-6500U CPU @ 2.50GHz' on 'Intel(R) OpenCL' at 0x1a553202dc0> succeeded, but said:

Device build started
Device build done
Reload Program Binary Object.
  warn(text, Compile

ValueError: shapes (40,1) and (25000,40) not aligned: 1 (dim 1) != 25000 (dim 0)

Notice that the computation could'nt been done on the CPU on the opposite it has been done on the GPU

## Conclusion

In [18]:
#We indeed obtain a square matrix (40,40) with the GPU, our variables size with a low computation duration.
#Let us display the variance covariance matrix, its' diagonal gives the empirical variance of each X. 
#It is the goal of the PCA to decide on which axes (2 dimensions) to project the data.

print(V)


[[  2.00978137e+11              inf              inf ...,   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  4.20389539e-44   0.00000000e+00   3.92363570e-44 ...,   5.89946653e-43
    3.25913779e+12   5.89946653e-43]
 [  1.40129846e-45   1.96181785e-44   4.20389539e-44 ...,   5.89946653e-43
    2.25132119e+14   5.89946653e-43]
 ..., 
 [  1.27378597e+14   5.89946653e-43   1.27379133e+14 ...,   5.89946653e-43
    1.27388797e+14   5.89946653e-43]
 [  1.27389334e+14   5.89946653e-43   1.27389871e+14 ...,   5.89946653e-43
    1.27399535e+14   5.89946653e-43]
 [  1.27400071e+14   5.89946653e-43   1.27400608e+14 ...,   5.89946653e-43
    1.27410809e+14   5.89946653e-43]]


Notice that I wasn't able to produce V with a classic CPU operation beacause of a data broadcast problem.
Let us see  the total computation duration for the GPU and CPU


In [21]:
#GPU
Temps_GPU= delta_t_GPU_transpo_diag + delta_t_GPU_transpo_a + delta_t_GPU_mul_1+ delta_t_GPU_transpo_W + delta_t_GPU_Centré + delta_t_GPU_Centré_Transpo + delta_t_GPU_mul_2 + delta_t_GPU_mul_3

#CPU
Temps_CPU= delta_t_CPU_transpo_diag + delta_t_CPU_transpo_a + delta_t_CPU_mul_1+ delta_t_CPU_transpo_W + delta_t_CPU_Centré + delta_t_CPU_Centré_Transpo + delta_t_CPU_mul_2 
 



In [22]:
print(Temps_GPU)
print(Temps_CPU)

0:00:32.520314
0:00:00.078136
